In [ ]:
%load_ext autoreload
%autoreload 2
import os
import os.path as osp
import torch, torchvision
import random
import numpy as np
import PIL.Image as PImage, PIL.ImageDraw as PImageDraw
import torch
import torch.nn
import argparse
import os
import numpy as np
from options import HiDDenConfiguration

import utils
from model.hidden import *
from noise_layers.noiser import Noiser
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

In [ ]:
base_path = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/"
device = torch.device('cuda:4')
device2 = torch.device('cuda:5')
message_length = 30

batch = 8

In [ ]:
def load_hidden(options_file_path,checkpoint_file_path,device):
    train_options, hidden_config, noise_config = utils.load_options(options_file_path)
    noiser = Noiser(noise_config,device)
    checkpoint = torch.load(checkpoint_file_path,weights_only=True)
    hidden_net = Hidden(hidden_config, device, noiser, None)
    utils.model_from_checkpoint(hidden_net, checkpoint)
    return hidden_net.encoder_decoder.encoder, hidden_net.encoder_decoder.decoder



def load_fhat(source_image_path,batch):
    fhat = torch.load(source_image,weights_only=True)
    print("Original Tensor Shape:", fhat.shape)
    if fhat.dim() == 3:
        fhat.unsqueeze_(0)
    if fhat.dim() == 4 and fhat.shape[0] > 10:
        fhat = fhat[:batch]
    fhat = fhat.to(device)
    print("Original Tensor Shape:", fhat.shape)
    return fhat
    

In [ ]:
def cal_error(decoded_message,message):
    
    decoded_rounded = decoded_message.detach().cpu().numpy().round().clip(0, 1)
    message_detached = message.detach().cpu().numpy()
    bitwise_avg_err = np.sum(np.abs(decoded_rounded - message.detach().cpu().numpy()))/(fhat.shape[0] * message.shape[1])
    return bitwise_avg_err

### Watermark

In [ ]:
# encoded_images = hidden_net.encoder_decoder.encoder(i,m)
def add_watermark(image,message,hidden_encoder,device):
    message = message.to(device)
    image = image.to(device)
    encoded_image = hidden_encoder(image,message)
    return encoded_image


def get_watermark(image,hidden_decoder,device):
    image = image.to(device)
    decoded_message  = hidden_decoder(image)
    return decoded_message 

In [ ]:
def repeat_test(test_func, num_trials=100):
    total_bitwise_avg_err = 0.0
    for i in range(num_trials):
        bitwise_avg_err = test_func()  
        total_bitwise_avg_err += bitwise_avg_err
    average_bitwise_avg_err = total_bitwise_avg_err / num_trials
    
    return average_bitwise_avg_err

In [ ]:
from VAR_model.var_use import VarTool
var = VarTool(device = device2)

source_image = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/images/epoch-original-300.pt"
fhat = load_fhat(source_image,8)



In [ ]:
base_path1 = "/home/yw699/codes/RESULT/E1 2025.02.03--19-43-00/"
options_file1 = os.path.join(base_path1,"options-and-config.pickle")
checkpoint_file1 =os.path.join(base_path1,"checkpoints/E1--epoch-300.pyt")
hidden_encoder1,hidden_decoder1 = load_hidden(options_file1,checkpoint_file1,device)



In [ ]:
base_path2 = "/home/yw699/codes/VAR-Watermark/runs/only-encoder 2025.02.07--10-58-15"
options_file2 = os.path.join(base_path2,"options-and-config.pickle")
checkpoint_file2 =os.path.join(base_path2,"only-encoder--epoch-88.pyt")
hidden_encoder2,hidden_decoder2 = load_hidden(options_file2,checkpoint_file2,device)

In [ ]:


def test_one_time_n():
    message = torch.Tensor(np.random.choice([0, 1], (fhat.shape[0],message_length))).to(device)
    fhat_watermark = add_watermark(fhat,message,hidden_encoder1,device)
    image_watermark = var.var_decoder(fhat_watermark)
    assert image_watermark.shape[1:] == (3, 256, 256), f"Expected shape (*, 3, 256, 256), but got {image_watermark.shape}"
    fhat_detect = var.var_encoder(image_watermark)
    watermark = get_watermark(fhat_detect,hidden_decoder1,device)
    bitwise_avg_err = cal_error(watermark,message).item()

    return bitwise_avg_err
test_one_time_n()

## Method1
直接检测fhat，用于测试 embedding hidden 的 performance

使用 fhat 训练的 hidden_encoder

使用 fhat 训练的 hidden_decoder 



In [ ]:
base_path = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/"
options_file = os.path.join(base_path,"options-and-config.pickle")
checkpoint_file =os.path.join(base_path,"checkpoints/watermark--epoch-300.pyt")
hidden_encoder,hidden_decoder = load_hidden(options_file,checkpoint_file,device)

source_image = os.path.join(base_path,"images/epoch-original-300.pt")
fhat = load_fhat(source_image,8)

In [ ]:
def test_one_time1():
    message = torch.Tensor(np.random.choice([0, 1], (fhat.shape[0],message_length))).to(device)
    fhat_watermark = add_watermark(fhat,message,hidden_encoder,device)
    watermark = get_watermark(fhat_watermark,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()
    return bitwise_avg_err

test_one_time1()

In [ ]:
repeat_test(test_one_time1, num_trials=100)

## Method2

- 使用 fhat 训练的 hidden_encoder

- 使用 fhat 训练的 hidden_decoder 

将 fhat 使用 hidden_encoder 加上 watermark
使用 var_decoder 生成 image

检测过程：

使用 var_encoder 生成 fhat

然后使用 hidden_decoder 获得 watermark





In [ ]:
from VAR_model.var_use import VarTool
var = VarTool(device = device2)

source_image = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/images/epoch-original-300.pt"
fhat = load_fhat(source_image,8)

In [ ]:
def test_one_time2():
    message = torch.Tensor(np.random.choice([0, 1], (fhat.shape[0],message_length))).to(device)
    fhat_watermark = add_watermark(fhat,message,hidden_encoder,device)
    image_watermark = var.var_decoder(fhat_watermark)
    assert image_watermark.shape[1:] == (3, 256, 256), f"Expected shape (*, 3, 256, 256), but got {image_watermark.shape}"
    fhat_detect = var.var_encoder(image_watermark)
    watermark = get_watermark(fhat_detect,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()

    return bitwise_avg_err
test_one_time2()

In [ ]:
repeat_test(test_one_time2, num_trials=100)
# 0.24970833256840705

## Method 3+4

这个hidden_encoder可以实现从fhat嵌入到image中的识别


In [ ]:
base_path = "/home/yw699/codes/VAR-Watermark/runs/E1 2025.02.03--19-43-00"
options_file = os.path.join(base_path,"options-and-config.pickle")
checkpoint_file =os.path.join(base_path,"checkpoints/E1--epoch-300.pyt")
hidden_encoder,hidden_decoder = load_hidden(options_file,checkpoint_file,device)

source_image = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/images/epoch-original-300.pt"
fhat = load_fhat(source_image,8)


In [ ]:
def test_one_time3():
    message = torch.Tensor(np.random.choice([0, 1], (fhat.shape[0],message_length))).to(device)
    fhat_watermark = add_watermark(fhat,message,hidden_encoder,device)
    watermark = get_watermark(fhat_watermark,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()
    return bitwise_avg_err

test_one_time3()

In [ ]:
repeat_test(test_one_time3, num_trials=100)
# 0.04029166753403843

In [ ]:
def test_one_time4():
    message = torch.Tensor(np.random.choice([0, 1], (fhat.shape[0],message_length))).to(device)
    #var.save_image(var.var_decoder(fhat),"00.png")
    fhat_watermark = add_watermark(fhat,message,hidden_encoder,device)
    image_watermark = var.var_decoder(fhat_watermark)
    assert image_watermark.shape[1:] == (3, 256, 256), f"Expected shape (*, 3, 256, 256), but got {image_watermark.shape}"
    #var.save_image(image_watermark,"4.png")
    fhat_detect = var.var_encoder(image_watermark)
    watermark = get_watermark(fhat_detect,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()

    return bitwise_avg_err
test_one_time4()

In [ ]:
repeat_test(test_one_time4, num_trials=100)
# 0.07483333380892873

## Method 5 + 6

在每个残差层使用hidden_encoder嵌入水印 + 最后在fhat上再加一遍

不知道这种robustness会不会更好



In [ ]:
class_labels = [1,3,5,7,100,2,10,8]
fhats,hs = var.generate_form_labels(class_labels)

assert torch.allclose(fhats[0], hs[0], atol=1e-6), f"fhats[0] != hs[0]"
assert torch.allclose(fhats[1], fhats[0] + hs[1], atol=1e-6), f"fhats[i+1] != fhats[i] + hs[i]"
assert torch.allclose(fhats[9], fhats[8] + hs[9], atol=1e-6), f"fhats[i+1] != fhats[i] + hs[i]"

image = var.var_decoder(fhats[-1])

var.save_image(image,"best.png")

In [ ]:
base_path = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/"
options_file = os.path.join(base_path,"options-and-config.pickle")
checkpoint_file =os.path.join(base_path,"checkpoints/watermark--epoch-300.pyt")
hidden_encoder,hidden_decoder = load_hidden(options_file,checkpoint_file,device)

# source_image = os.path.join(base_path,"images/epoch-original-300.pt")
# fhat = load_fhat(source_image,10)


In [ ]:
def test_one_time5():
    message = torch.Tensor(np.random.choice([0, 1], (fhats[-1].shape[0],message_length))).to(device)
    hs9_watermark = add_watermark(hs[-1].clone(),message,hidden_encoder,device)
    fhat_watermark = hs9_watermark + fhats[8].clone().to(device)
    fhat_watermark = add_watermark(fhat_watermark,message,hidden_encoder,device)
    watermark = get_watermark(fhat_watermark,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()
    return bitwise_avg_err

test_one_time5()


In [ ]:
repeat_test(test_one_time5, num_trials=100)

In [ ]:
def test_one_time6():
    message = torch.Tensor(np.random.choice([0, 1], (fhats[-1].shape[0],message_length))).to(device)
    hs9_watermark = add_watermark(hs[-1].clone(),message,hidden_encoder,device)
    fhat_watermark = hs9_watermark + fhats[8].clone().to(device)
    fhat_watermark = add_watermark(fhat_watermark,message,hidden_encoder,device)
    image_watermark = var.var_decoder(fhat_watermark)
    assert image_watermark.shape[1:] == (3, 256, 256), f"Expected shape (*, 3, 256, 256), but got {image_watermark.shape}"
    fhat_detect = var.var_encoder(image_watermark)
    watermark = get_watermark(fhat_detect,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()

    return bitwise_avg_err
test_one_time6()

In [ ]:
repeat_test(test_one_time6, num_trials=100)

In [ ]:
base_path = "/home/yw699/codes/VAR-Watermark/runs/E1 2025.02.03--19-43-00"
options_file = os.path.join(base_path,"options-and-config.pickle")
checkpoint_file =os.path.join(base_path,"checkpoints/E1--epoch-300.pyt")
hidden_encoder,hidden_decoder = load_hidden(options_file,checkpoint_file,device)

source_image = "/home/yw699/codes/RESULT/watermark 2024.12.17--15-42-23/images/epoch-original-300.pt"
fhat = load_fhat(source_image,8)

In [ ]:
def test_one_time7():
    message = torch.Tensor(np.random.choice([0, 1], (fhats[-1].shape[0],message_length))).to(device)
    hs9_watermark = add_watermark(hs[-1].clone(),message,hidden_encoder,device)
    fhat_watermark = hs9_watermark + fhats[8].clone().to(device)
    fhat_watermark = add_watermark(fhat_watermark,message,hidden_encoder,device)
    watermark = get_watermark(fhat_watermark,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()
    return bitwise_avg_err

test_one_time7()

In [ ]:
repeat_test(test_one_time7, num_trials=100)

In [ ]:
def test_one_time8():
    message = torch.Tensor(np.random.choice([0, 1], (fhats[-1].shape[0],message_length))).to(device)
    hs9_watermark = add_watermark(hs[-1].clone(),message,hidden_encoder,device)
    fhat_watermark = hs9_watermark + fhats[8].clone().to(device)
    fhat_watermark = add_watermark(fhat_watermark,message,hidden_encoder,device)
    image_watermark = var.var_decoder(fhat_watermark)
    assert image_watermark.shape[1:] == (3, 256, 256), f"Expected shape (*, 3, 256, 256), but got {image_watermark.shape}"
    fhat_detect = var.var_encoder(image_watermark)
    watermark = get_watermark(fhat_detect,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()

    return bitwise_avg_err
test_one_time8()

In [ ]:
repeat_test(test_one_time8, num_trials=100)

In [ ]:
def test_one_time9():
    message = torch.Tensor(np.random.choice([0, 1], (fhats[-1].shape[0],message_length))).to(device)
    hs9_watermark = add_watermark(hs[-1].clone(),message,hidden_encoder,device)
    hs8_watermark = add_watermark(hs[-2].clone(),message,hidden_encoder,device)
    fhat_watermark = hs9_watermark + hs8_watermark + fhats[7].clone().to(device)
    fhat_watermark = add_watermark(fhat_watermark,message,hidden_encoder,device)
    image_watermark = var.var_decoder(fhat_watermark)
    assert image_watermark.shape[1:] == (3, 256, 256), f"Expected shape (*, 3, 256, 256), but got {image_watermark.shape}"
    fhat_detect = var.var_encoder(image_watermark)
    watermark = get_watermark(fhat_detect,hidden_decoder,device)
    bitwise_avg_err = cal_error(watermark,message).item()

    return bitwise_avg_err
test_one_time9()

In [ ]:
repeat_test(test_one_time9, num_trials=100)

## Method 7

image-f-fhat(codebook)

或者在vq之前就提前处理这个value然后加水印，分析一下codebook按理说应该是离散的几个值

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


data = {
    "Method": [1, 2, 3, 4, 5, 6, 7, 8],
    "Bitwise_a": [0.01825, 0.249708, 0.040291, 0.073333, 0.021416, 0.221000, 0.03825, 0.11],
    "Hidden-encoder/decoder": [
        "Directly Detect",
        "Use VAR Encoder/Decoder",
        "Directly Detect",
        "Use VAR Encoder/Decoder",
        "Directly Detect",
        "Use VAR Encoder/Decoder",
        "Directly Detect",
        "Use VAR Encoder/Decoder",
    ]
}

df = pd.DataFrame(data)


bar_width = 0.4
x = np.arange(len(df) // 2)  


fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - bar_width / 2, df["Bitwise_a"][::2], width=bar_width, color="blue", label="Directly Detect")
bars2 = ax.bar(x + bar_width / 2, df["Bitwise_a"][1::2], width=bar_width, color="orange", label="Use VAR Encoder/Decoder")


for bar in bars1 + bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.2f}", ha="center", va="bottom", fontsize=10)


ax.set_xticks(x)
ax.set_xticklabels([f"{df['Method'][i]} & {df['Method'][i+1]}" for i in range(0, len(df), 2)])
ax.set_xlabel("Method Pairs")
ax.set_ylabel("Bitwise_avg_err")
ax.set_title("Method vs Bitwise_avg_err")


ax.set_ylim(0, 0.5)
ax.legend()

plt.show()
